In [ ]:
import pandas as pd

In [ ]:

#NLTK
import nltk
import nltk.corpus 
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:

import DataPrep
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import  LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix, f1_score, classification_report

In [ ]:
train = pd.read_csv('./dataset/train.csv')
test = pd.read_csv('./dataset/test.csv')
val = pd.read_csv('./dataset/val.csv')

train = DataPrep.map_labels(train)
val = DataPrep.map_labels(val)
test = DataPrep.map_labels(test)


In [ ]:
train = pd.concat([train,val], ignore_index= True)

In [ ]:
def text_process(mess):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Now just remove any stopwords
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [ ]:
# Bag of words
bow_transformer = CountVectorizer(analyzer= text_process).fit(train['statement'])
print(len(bow_transformer.vocabulary_))

15557


In [ ]:
# Naive Baye's

NaiveB = Pipeline([
                   ('bow', CountVectorizer(analyzer=text_process, lowercase=True)),
                   ('classifier', MultinomialNB()),
])

In [ ]:
NaiveB.fit(train['statement'], train['label'])

Pipeline(memory=None,
         steps=[('bow',
                 CountVectorizer(analyzer=<function text_process at 0x7f3532c2f8c8>,
                                 binary=False, decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('classifier',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [ ]:
NaiveB_predicted = NaiveB.predict(test['statement'])
print(classification_report(test['label'], NaiveB_predicted))
print('\n')
print(confusion_matrix(test['label'], NaiveB_predicted))

              precision    recall  f1-score   support

           0       0.57      0.46      0.51       549
           1       0.64      0.73      0.68       709

    accuracy                           0.61      1258
   macro avg       0.60      0.59      0.59      1258
weighted avg       0.61      0.61      0.60      1258



[[252 297]
 [192 517]]


In [ ]:
#logistic Regression
logR = Pipeline([
                 ('bow', CountVectorizer(analyzer=text_process, lowercase=True)),
                 ('logR',LogisticRegression())
])

In [ ]:
logR.fit(train['statement'], train['label'])

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Pipeline(memory=None,
         steps=[('bow',
                 CountVectorizer(analyzer=<function text_process at 0x7f3532c2f8c8>,
                                 binary=False, decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('logR',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=100,
                                    multi_class='auto', n_jo

In [ ]:
logR_predicted = logR.predict(test['statement'])
print(classification_report(test['label'], logR_predicted))
print('\n')
print(confusion_matrix(test['label'], logR_predicted)) 

              precision    recall  f1-score   support

           0       0.55      0.49      0.52       549
           1       0.63      0.68      0.66       709

    accuracy                           0.60      1258
   macro avg       0.59      0.59      0.59      1258
weighted avg       0.60      0.60      0.60      1258



[[270 279]
 [224 485]]


In [ ]:
#Random forest

randomForest = Pipeline([
                         ('bow', CountVectorizer(analyzer=text_process, lowercase=True)),
                         ('randForestCls',RandomForestClassifier(n_estimators=200))
])

In [ ]:
randomForest.fit(train['statement'], train['label'])

Pipeline(memory=None,
         steps=[('bow',
                 CountVectorizer(analyzer=<function text_process at 0x7f3532c2f8c8>,
                                 binary=False, decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b...
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.

In [ ]:
randomForest_predicted = randomForest.predict(test['statement'])
print(classification_report(test['label'], randomForest_predicted))
print('\n')
print(confusion_matrix(test['label'], randomForest_predicted)) 

              precision    recall  f1-score   support

           0       0.57      0.43      0.49       549
           1       0.63      0.74      0.68       709

    accuracy                           0.61      1258
   macro avg       0.60      0.59      0.59      1258
weighted avg       0.60      0.61      0.60      1258



[[238 311]
 [182 527]]


In [ ]:
#svm classifier
svm_ = Pipeline([
                ('bow', CountVectorizer(analyzer=text_process, lowercase=True)),
                ('svm_clsf', svm.LinearSVC()),
                
])

In [ ]:
svm_.fit(train['statement'], train['label'])

Pipeline(memory=None,
         steps=[('bow',
                 CountVectorizer(analyzer=<function text_process at 0x7f3532c2f8c8>,
                                 binary=False, decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('svm_clsf',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
                           loss='squared_hinge', max_iter=1000,
                           multi_class='ovr', penalty='l2', random_state=Non

In [ ]:
svm_predicted = svm_.predict(test['statement'])
print(classification_report(test['label'], svm_predicted))
print('\n')
print(confusion_matrix(test['label'], svm_predicted)) 


              precision    recall  f1-score   support

           0       0.51      0.51      0.51       549
           1       0.62      0.63      0.62       709

    accuracy                           0.57      1258
   macro avg       0.57      0.57      0.57      1258
weighted avg       0.57      0.57      0.57      1258



[[278 271]
 [265 444]]


In [ ]:
sgd = Pipeline([
                ('bow', CountVectorizer(analyzer=text_process, lowercase=True)),
                ('sgd', SGDClassifier())
])

In [ ]:
sgd.fit(train['statement'], train['label'])

Pipeline(memory=None,
         steps=[('bow',
                 CountVectorizer(analyzer=<function text_process at 0x7f3532c2f8c8>,
                                 binary=False, decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b...
                 SGDClassifier(alpha=0.0001, average=False, class_weight=None,
                               early_stopping=False, epsilon=0.1, eta0=0.0,
                               fit_intercept=True, l1_ratio=0.15,
                               learning_rate='optimal', loss='hinge',
                               max_iter=1000, n_iter_no_change=5, n_jobs=None

In [ ]:
sgd_predicted = sgd.predict(test['statement'])
print(classification_report(test['label'], sgd_predicted))
print('\n')
print(confusion_matrix(test['label'], sgd_predicted)) 


              precision    recall  f1-score   support

           0       0.54      0.50      0.52       549
           1       0.63      0.68      0.65       709

    accuracy                           0.60      1258
   macro avg       0.59      0.59      0.59      1258
weighted avg       0.59      0.60      0.60      1258



[[272 277]
 [229 480]]
